This jupyter notebook is to show function `babs-init`.

In [1]:
# generate html file from this ipynb: 
# 1. set up the terminal:
#   $ source ~/.bashrc   # freesurfer and templateflow env variables
#   $ conda activate mydatalad
# 2. manually install BABS:
#   $ pip install -e .
# 3. then run:
#   $ jupyter nbconvert --execute --to html notebooks/show_babs_init_InputBIDS.ipynb

# Interactively test out the bootstrapped scripts:
#   - open `participant_job.sh` and `submit_jobs.sh`
#   - enter the arguments value into terminal based on `submit_job.sh`
#   - $ where_to_run="/cbica/comp_space/$(basename $HOME)"   # using comp_space
# After the job is gone from the list, check:
#   - output_ria/xxx/xxx-xxx-xxx-xxx: $ git branch -a  # to see if the job is success
#   - `analysis/logs` for logs
#   - cd "/cbica/comp_space/$(basename $HOME)"   # comp_space to find out that job
#   - qaccj -j <jobID> for diagnosis
# Clone out:
#   datalad clone ria+file:///path/to/<container>/output_ria#~data singleORmulti-ses_<container>_outputs
# If you want to change `project_root`'s foldername, also change `output_ria/alias/data` symlink:
    # cd output_ria/alias
    # ln -sf /new/full/path/to/output_ria/xxx/xxx-xxx-xxx-xxx data

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs
 
# import sys
import os
import os.path as op
# sys.path.append( os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.path.abspath('')))), "babs"  ))

# from babs.core_functions import babs_init
# from babs.cli import *


There are several arguments when calling `babs-init`:

In [2]:
# This notebook only accepts qsiprep or fmriprep (input ds: BIDS)
# ++++++++++++++++++++++++++++++++++
flag_instance = "toybidsapp"
type_session = "multi-ses"
flag_where = "cubic"     # "cubic" or "local"
# ++++++++++++++++++++++++++++++++++
where_notebooks = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper/babs/notebooks"

# where:
if flag_where == "cubic":
    where_root = "/cbica/projects/BABS"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "local":
    where_root = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
else:
    raise Exception("not valid `flag_where`!")

if flag_instance == "toybidsapp":
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "j854e")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "zd9a6")

    input_cli = [["BIDS", input_ds]]
    project_name = "test_babs_" + type_session + "_" + flag_instance
    bidsapp = "toybidsapp"
    container_name = bidsapp + "-0-0-5"

elif flag_instance == "fmriprep":
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "j854e")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "zd9a6")

    project_name = "test_babs_" + type_session + "_fmriprep"
    bidsapp = "fmriprep"
    container_name = bidsapp + "-0-0-0"  # TODO: to change

elif flag_instance == "qsiprep":
    project_name = "test_babs_" + type_session + "_qsiprep"
    bidsapp = "qsiprep"
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "j854e")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "zd9a6")
    
    container_name = bidsapp + "-0-0-0"  # TODO: to change

else:
    raise Exception("`flag_instance` is not QSIPrep or fMRIPrep!")

container_ds = op.join(where_project, "toybidsapp-container")  
# TODO: to change this ^^ before running fmriprep/qsiprep on cubic!
if flag_where == "local":
    container_ds += "-docker"   # add "docker" at the end
container_config_yaml_file = op.join(where_notebooks, "example_container_" + flag_instance + ".yaml")

# list_sub_file = op.join(where_notebooks, "initial_sub_list_" + type_session + ".csv")
list_sub_file = None

Let's pass these arguments into `babs-init` CLI:

In [3]:
cmd = "babs-init \\\n"
cmd += "\t" + "--where_project " + where_project + " \\\n"
cmd += "\t" + "--project_name " + project_name + " \\\n"
cmd += "\t" + "--input " + "BIDS" + " " + input_ds + " \\\n"
if list_sub_file is not None:
    cmd += "\t" + "--list_sub_file " + list_sub_file + " \\\n"
cmd += "\t" + "--container_ds " + container_ds + " \\\n"
cmd += "\t" + "--container_name " + container_name + " \\\n"
cmd += "\t" + "--container_config_yaml_file " + container_config_yaml_file + " \\\n"
cmd += "\t" + "--type_session " + type_session + " \\\n"
cmd += "\t" + "--type_system " + "sge"

print("The command to execute:")
print(cmd)

The command to execute:
babs-init \
	--where_project /cbica/projects/BABS/data \
	--project_name test_babs_multi-ses_toybidsapp \
	--input BIDS /cbica/projects/BABS/data/j854e \
	--container_ds /cbica/projects/BABS/data/toybidsapp-container \
	--container_name toybidsapp-0-0-5 \
	--container_config_yaml_file /cbica/projects/BABS/babs/notebooks/example_container_toybidsapp.yaml \
	--type_session multi-ses \
	--type_system sge


In [ ]:
!($cmd)